In [49]:
%%writefile mapper.py
import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open('stop_words_en.txt') as f:
    stop_words = set(f.read().split())

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
        
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    words = [x.lower() for x in words if x.lower() not in stop_words]
    words_set = set(words)
    
    num_words = len(words)
    counter = Counter(words)
    article_id = int(article_id)
    for word in words:
        frequency = counter[word]
        tf = frequency / float(num_words)
        print "%s\t%d\t%f" % (word, article_id, tf)


Overwriting mapper.py


In [50]:
%%writefile reducer.py
from __future__ import division
import sys
from math import log

current_word = None
article_dict = {}

for line in sys.stdin:
    try:
        word, article_id, tf = line.strip().split('\t')
        article_id = int(article_id)
        tf = float(tf)
    except ValueError as e:
        continue
    
    if current_word != word:
        if current_word:
            idf = 1 / log(1 + len(article_dict))
            for key_article_id, tf in article_dict.iteritems():
                tfidf = tf * idf
                print "%s\t%d\t%f" % (current_word, key_article_id, tfidf)
        article_dict = {}
        current_word = word
    article_dict[article_id] = tf

if current_word:
    print "%s\t%d\t%f" % (current_word, article_id, tfidf)

Overwriting reducer.py


In [52]:
%%bash

OUT_DIR="tfidf_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming tfidf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat  ${OUT_DIR}/* | grep -P 'labor\t12\t' | cut -f3


0.000351


rm: `tfidf_result_1529467845522776': No such file or directory
18/06/20 04:10:47 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/06/20 04:10:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/06/20 04:10:48 INFO mapred.FileInputFormat: Total input files to process : 1
18/06/20 04:10:49 INFO mapreduce.JobSubmitter: number of splits:2
18/06/20 04:10:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1529463856073_0010
18/06/20 04:10:49 INFO impl.YarnClientImpl: Submitted application application_1529463856073_0010
18/06/20 04:10:49 INFO mapreduce.Job: The url to track the job: http://d958f05bfa46:8088/proxy/application_1529463856073_0010/
18/06/20 04:10:49 INFO mapreduce.Job: Running job: job_1529463856073_0010
18/06/20 04:10:54 INFO mapreduce.Job: Job job_1529463856073_0010 running in uber mode : false
18/06/20 04:10:54 INFO mapreduce.Job:  map 0% reduce 0%
18/06/20 04:11:09 INFO mapreduce.Job:  map 76% reduce 0%
18/06/20 04:11:10